In [1]:
import pickle
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm.notebook import trange

import matplotlib as mpl
import matplotlib.pyplot as plt

from function.omaseq import FetchOmaSeqBatch
from function.omaseq import TaxSeqFilter
from function.alignment import Alignment
from function.utilities import get_subset
from function.seqfilter import FastaExtreFilter

# Param

In [2]:
#####CHANGE HERE#####
uniprot_id = "Q9Y4C8"
#####CHANGE HERE#####

In [3]:
oma_path = Path("./output/fasta/a_oma")
grouped_path = Path("./output/fasta/b_grouped")
alied_path = Path("./output/fasta/c_alied")
extre_filtered_path = Path("./output/fasta/d_extre_filtered")

gap_filter_num = 20
max_filter_seq = 3

tax_ids = [7711, 7742, 32523, 40674] #chordata, vertebrata, tetrapoda, mammalia

# Fetch oma seqence

In [4]:
#get paralogs by uniprot id from OMA, 
#https://omabrowser.org/oma/home/
fetchomaseq = FetchOmaSeqBatch()
fetchomaseq.get_oma_seq(uniprot_id, oma_path)

# Group by taxonomy id

In [5]:
#filter paralogs by taxonomy id, and save as fasta file
t = trange(len(tax_ids), leave=True)
for i in t:
    t.set_description(str(tax_ids[i]))
    t.refresh()

    oma_fasta_path = oma_path / "{}.fasta".format(uniprot_id)

    grouped_tax_path = grouped_path / str(tax_ids[i])
    grouped_tax_path.mkdir(parents=True, exist_ok=True)
    grouped_fasta_path = grouped_tax_path / "{}.fasta".format(uniprot_id)

    taxfilter = TaxSeqFilter(tax_ids[i])
    taxfilter.taxfilter(oma_fasta_path, grouped_fasta_path)

  0%|          | 0/4 [00:00<?, ?it/s]

# Alignment

In [50]:
#alignment func by Biopython ClustalOmegaCommandline
#need to install ClustalOmega: http://www.clustal.org/omega/
alignment = Alignment()
t = trange(len(tax_ids), leave=True)
for i in t:
    t.set_description(str(tax_ids[i]))
    t.refresh()

    grouped_tax_path = grouped_path / str(tax_ids[i])
    grouped_fasta_path = grouped_tax_path / "{}.fasta".format(uniprot_id)

    alied_tax_path = alied_path / str(tax_ids[i])
    alied_tax_path.mkdir(parents=True, exist_ok=True)
    alied_fasta_path = alied_tax_path / "{}.fasta".format(uniprot_id)

    alignment.alignment_single(grouped_fasta_path, alied_fasta_path)

  0%|          | 0/4 [00:00<?, ?it/s]

# Extre filter

In [4]:
#filter some special paralogs, 
#i.e. some sequences in paralogs has long gap(>20) in alied fasta file, 
#while other sequences do not have gap, we think this is "special", and filter it
fastaextrefilter = FastaExtreFilter()
t = trange(len(tax_ids), leave=True)
for i in t:
    t.set_description(str(tax_ids[i]))
    t.refresh()

    alied_tax_path = alied_path / str(tax_ids[i])
    alied_fasta_path = alied_tax_path / "{}.fasta".format(uniprot_id)

    extre_filtered_tax_path = extre_filtered_path / str(tax_ids[i])
    extre_filtered_tax_path.mkdir(parents=True, exist_ok=True)
    extre_filtered_tax_fasta_path = extre_filtered_tax_path / "{}.fasta".format(uniprot_id)

    extre_index = fastaextrefilter.fasta_extre_filter(alied_fasta_path,
                                                      extre_filtered_tax_fasta_path,
                                                      gap_filter_num=gap_filter_num,
                                                      max_filter_seq=max_filter_seq,
                                                      )
    print(extre_index)
    

  0%|          | 0/4 [00:00<?, ?it/s]

{'before': 109, 'after': 98, 'removed_sequence_index': [108, 104, 93, 90, 74, 67, 66, 48, 37, 14, 1]}
{'before': 107, 'after': 100, 'removed_sequence_index': [103, 92, 89, 73, 66, 65, 36]}
{'before': 75, 'after': 65, 'removed_sequence_index': [71, 60, 57, 41, 34, 33, 32, 28, 15, 4]}
{'before': 59, 'after': 52, 'removed_sequence_index': [57, 41, 34, 33, 32, 15, 4]}


In [4]:
alied_path = Path("/home/wenlin/d/rbp/oma_all/d_alied_rbp")
extre_filtered_path = Path("/home/wenlin/d/rbp/oma_all/e_extre_filtered_rbp")

In [5]:
fastaextrefilter = FastaExtreFilter()
t = trange(len(tax_ids), leave=True)
for i in t:
    t.set_description(str(tax_ids[i]))
    t.refresh()

    alied_tax_path = alied_path / str(tax_ids[i])
    

    extre_filtered_tax_path = extre_filtered_path / str(tax_ids[i])
    extre_filtered_tax_path.mkdir(parents=True, exist_ok=True)
    
    for alied_fasta_path in alied_tax_path.rglob("*.fasta"):
        try:
            uniprot_id = alied_fasta_path.parts[-1].split('.')[0]

            extre_filtered_tax_fasta_path = extre_filtered_tax_path / "{}.fasta".format(uniprot_id)


            extre_index = fastaextrefilter.fasta_extre_filter(alied_fasta_path,
                                                              extre_filtered_tax_fasta_path,
                                                              gap_filter_num=gap_filter_num,
                                                              max_filter_seq=max_filter_seq,
                                                              )
        except:
            print("{}".format(uniprot_id))

  0%|          | 0/4 [00:00<?, ?it/s]

Q6P1Q9 FILTERED FASTA IS EMPTY
Q6P1Q9
Q13117 FILTERED FASTA IS EMPTY
Q13117
P0DJD3 FILTERED FASTA IS EMPTY
P0DJD3
P17844 FILTERED FASTA IS EMPTY
P17844
Q9NR90 FILTERED FASTA IS EMPTY
Q9NR90
P49792 FILTERED FASTA IS EMPTY
P49792
Q6P1Q9 FILTERED FASTA IS EMPTY
Q6P1Q9
Q13117 FILTERED FASTA IS EMPTY
Q13117
P0DJD3 FILTERED FASTA IS EMPTY
P0DJD3
Q8N5A5 FILTERED FASTA IS EMPTY
Q8N5A5
P17844 FILTERED FASTA IS EMPTY
P17844
Q9NR90 FILTERED FASTA IS EMPTY
Q9NR90
P49792 FILTERED FASTA IS EMPTY
P49792
Q6P1Q9 FILTERED FASTA IS EMPTY
Q6P1Q9
Q9BZB8 FILTERED FASTA IS EMPTY
Q9BZB8
P0DJD3 FILTERED FASTA IS EMPTY
P0DJD3
Q8N5A5 FILTERED FASTA IS EMPTY
Q8N5A5
P17844 FILTERED FASTA IS EMPTY
P17844
B5ME19 FILTERED FASTA IS EMPTY
B5ME19
Q9NR90 FILTERED FASTA IS EMPTY
Q9NR90
P49792 FILTERED FASTA IS EMPTY
P49792
Q9Y6E2 FILTERED FASTA IS EMPTY
Q9Y6E2
Q6IS14 FILTERED FASTA IS EMPTY
Q6IS14
Q6P1Q9 FILTERED FASTA IS EMPTY
Q6P1Q9
Q05639 FILTERED FASTA IS EMPTY
Q05639
Q8IZ69 FILTERED FASTA IS EMPTY
Q8IZ69
Q9BZB8 FILTE